# Multi-pedestrian Dataset Tutorial

Author: [Zhe Huang](https://github.com/tedhuang96) <br/>
Date: Nov. 8, 2020 <br/>
The code base is heavily borrowed from [Social-STGCNN](https://github.com/abduallahmohamed/Social-STGCNN).

## 1. Set up source files, packages and arguments.

In [ ]:
# Clone source files into Colab.
!git clone https://github.com/tedhuang96/torch_data_tutorial.git

In [ ]:
# Import packages and check GPU.
# 
### expected output: ###
### cuda:0 ### or ### cpu ###

import torch
import argparse
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from os import listdir
from os.path import join
from IPython.display import HTML
from matplotlib import animation, rc
from IPython.display import HTML

from torch_data_tutorial.utils import anorm, create_datasets
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Write arguments.
# 
### expected output: ###
### Namespace(attn_mech='auth', dataset='zara1', obs_seq_len=8, pred_seq_len=12) ###

def arg_parse_notebook(obs_seq_len, pred_seq_len, dataset, attn_mech):
    obs_seq_len, pred_seq_len = str(obs_seq_len), str(pred_seq_len)
    parser = argparse.ArgumentParser()
    parser.add_argument('--obs_seq_len', type=int, default=0)
    parser.add_argument('--pred_seq_len', type=int, default=0)
    parser.add_argument('--dataset', default='eth',
                        help='eth,hotel,univ,zara1,zara2')
    parser.add_argument('--attn_mech', default='glob_kip',
                        help='attention mechanism: glob_kip, auth')
    args_list = ['--obs_seq_len', obs_seq_len, '--pred_seq_len', pred_seq_len, '--dataset', dataset, '--attn_mech', attn_mech]
    return parser.parse_args(args_list)

obs_seq_len, pred_seq_len, dataset, attn_mech = 8, 12, 'zara1', 'auth'
args = arg_parse_notebook(obs_seq_len, pred_seq_len, dataset, attn_mech)
print(args)

## 2. Introduce multi-pedestrian datasets.

Below are snapshots from ETH/UCY datasets. These datasets provide a [benchmark](http://trajnet.stanford.edu/) for multi-pedestrian trajectory prediction algorithms which are focused on pedestrian interaction mechanism. ETH contains two scenarios and UCY contains other three. They recorded and annotated pedestrian trajectories from a bird's eye view. The annotations include frame_id, pedestrian_id, x, y. 
![title](multi_pedestrian_scenarios.png)

In [ ]:
# Load an example dataset into pandas.
# 
### expected output: ###
### filename:  torch_data_tutorial/datasets/zara1/train/train.txt ###
### data table: <traj_data_table> ###
pkg_path = 'torch_data_tutorial'
traj_data_folder = join(pkg_path, 'datasets', args.dataset, 'train') # may change 'datasets' to 'datasets_loo' for the leave-one-out setting.
traj_data_filepath = join(traj_data_folder, listdir(traj_data_folder)[0])
traj_data_table = pd.read_csv(traj_data_filepath, sep="\t", header=None)
traj_data_table.columns = ["frame_id", "ped_id", "x", "y"]
print('filename: ', traj_data_filepath)
print('data table: ')
print(traj_data_table)

In [ ]:
# Plot a pedestrian's trajectory.
# 
### expected output: ###
### <ped_traj> ###
### A trajectory plot corresponding to the pedestrian we queried. ###

ped_traj = traj_data_table.loc[traj_data_table['ped_id'] == 1]
print(ped_traj)
_, ax = plt.subplots()
ax.plot(ped_traj['x'], ped_traj['y'], 'o-')
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# Plot pedestrian positions in a frame.
# 
### expected output: ###
### <frame_pos> ###
### A scatter plot with each point representing a pedestrian position in that specific frame. ###

frame_pos = traj_data_table.loc[traj_data_table['frame_id']==0]
print(frame_pos)
_, ax = plt.subplots()
ax.plot(frame_pos['x'], frame_pos['y'], 'o')
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# Animation of pedestrian motions in a 20-frame period.
# 
### expected output: ###
### A scatter plot animation with each point representing a pedestrian position. ###
### blue means the motion is during observation period, and ren means the motion is during prediction period. ###

fig, ax = plt.subplots()
ax.set_xlim((0, 16))
ax.set_ylim((2, 9))
ped_pos, = ax.plot([], [],'o')
plt.close()

def init():
    ped_pos.set_data([], [])
    return (ped_pos,)

def animate(i):
    frame_pos = traj_data_table.loc[traj_data_table['frame_id'] == i*10]
    x, y = frame_pos['x'], frame_pos['y']
    ped_pos.set_data(x, y)
    if i > 8:
        ped_pos.set_color('C3') # red
    return (ped_pos,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=20, interval=200, blit=True)
HTML(anim.to_html5_video())

In [ ]:
# We are interested in predicting pedestrian motion in the later 12 time steps
# given the observation in the first 8 time steps.
#
# We filter out common pedestrians across all 20 time steps,
# and their trajectories in these 20 time steps formed a data sample.
# 
### expected output: ###
### ped_ids shown up in all 20 frames: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0] ###
### Two trajectory plots ###

ped_ids_20_frames = []
for i in range(0, 19):
    ped_ids_20_frames.append(set(traj_data_table.loc[traj_data_table['frame_id'] == i*10]['ped_id']))
common_ped_ids_20_frames = list(set.intersection(*ped_ids_20_frames))
print('ped_ids shown up in all 20 frames:', common_ped_ids_20_frames)


_, ax = plt.subplots()
for common_ped_id in common_ped_ids_20_frames[:8]: 
    ped_traj_pred = traj_data_table.loc[(traj_data_table['ped_id'] == common_ped_id) & (traj_data_table['frame_id'] <= 190)]
    ax.plot(ped_traj_pred['x'], ped_traj_pred['y'], 'o-')
ax.set_aspect('equal', adjustable='box')
plt.show()


_, ax = plt.subplots()
for common_ped_id in common_ped_ids_20_frames[:8]: 
    ped_traj_pred = traj_data_table.loc[(traj_data_table['ped_id'] == common_ped_id) & (traj_data_table['frame_id'] >= 70) \
                                       & (traj_data_table['frame_id'] <= 190)]
    ax.plot(ped_traj_pred['x'], ped_traj_pred['y'], 'o-', c='C3') # red
    ped_traj_obs = traj_data_table.loc[(traj_data_table['ped_id'] == common_ped_id) & (traj_data_table['frame_id'] < 80)]
    ax.plot(ped_traj_obs['x'], ped_traj_obs['y'], 'o-',c='C0') # blue
ax.set_aspect('equal', adjustable='box')
plt.show()



## 3. Customize a torch dataset for multi-pedestrian trajectory data.

Deep learning has become increasingly effective in learning patterns from data in recent years. We will use [PyTorch](https://pytorch.org) as our deep learning framework to handle multi-pedestrian trajectory prediction tasks.

The first step to accomplish the task is to build a dataset that algorithms developed in Pytorch can recognize as input. Check out TrajectoryDataset class in utils.py, which we will be using to create the dataset.

Extension: [Mini-batch Gradient Descent](https://xzz201920.medium.com/gradient-descent-stochastic-vs-mini-batch-vs-batch-vs-adagrad-vs-rmsprop-vs-adam-3aa652318b0d).

![title](sgd_bgd_mbgd.png)


In [ ]:
### expected output: ###
# ['torch_data_tutorial/datasets/zara1/train/train.txt']
# Processing Data .....
# 100%
# 503/503 [00:11<00:00, 42.84it/s]
#
# ['torch_data_tutorial/datasets/zara1/val/val.txt']
# Processing Data .....
# 100%
# 85/85 [00:05<00:00, 15.75it/s]
#
# ['torch_data_tutorial/datasets/zara1/test/crowds_zara01.txt']
# Processing Data .....
# 100%
# 602/602 [00:13<00:00, 43.06it/s]

pkg_path = 'torch_data_tutorial'
dsets = create_datasets(args, pkg_path, save_datasets=True)
dloader_train = DataLoader(
        dsets['train'],
        batch_size=1,
        shuffle=True,
        num_workers=1)

In [ ]:
# Define functions for visualization.

def get_batch_sample(loader_test, device='cuda:0'):
    batch_count = 1
    for cnt, batch in enumerate(loader_test):
        batch_count += 1
        # Get data
        batch = [tensor.to(device) for tensor in batch]
        # ** Name of variables in a batch
        # * obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
        # *    loss_mask, V_obs, A_obs, V_tr, A_tr = batch
        var_names = [
            'obs_traj', 'pred_traj_gt', 'obs_traj_rel', 'pred_traj_gt_rel', 'non_linear_ped',
            'loss_mask', 'V_obs', 'A_obs', 'V_tr', 'A_tr']
        for var_name_i, batch_i in zip(var_names, batch):
            if var_name_i == 'obs_traj' and batch_i.shape[1] == 5: # get five-pedestrian case
                return batch

def visualize_dataloader(data_loader, attn_scale=1., device='cuda:0'):
    """
    visualize data and attention in a batch generated by data_loader.
    """
    batch = get_batch_sample(data_loader, device=device)
    obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
        loss_mask, V_obs, A_obs, V_tr, A_tr = batch
    obs_ts = 2 # time of attention
    attn = A_obs[0, obs_ts]  # (num_peds, num_peds)
    fig, ax = plt.subplots()
    colors = 'rgggg'
#     attn_scale = 1. # size of attention circle
    for ped_i in range(obs_traj.shape[1]):
        x_obs_i = obs_traj[0, ped_i].to('cpu')
        x_gt_i = pred_traj_gt[0, ped_i].to('cpu')
        x_concat_i = torch.cat([x_obs_i, x_gt_i], dim=1)
        ax.plot(x_concat_i[0, :], x_concat_i[1, :], 'o-',c='C3')
        ax.plot(x_obs_i[0, :], x_obs_i[1, :], 'o-', c='C0')
        ax.plot(x_obs_i[0, obs_ts], x_obs_i[1, obs_ts], 'ko-')
        attn_i = plt.Circle(
            x_obs_i[:, obs_ts], abs(attn[ped_i, 0])*attn_scale, color=colors[ped_i], fill=False)
        ax.add_artist(attn_i)
    ax.set_aspect('equal', adjustable='box')
    plt.show()
    
def dataset_format():
    """
    Documentation on structure of a batch from the dataset.
    - batch # list.

        - obs_traj # global positions in observation. # tensor: (1, num_peds, 2, 8)
        # 2 means x and y. # 8 means obs_period.

        - pred_traj_gt # ground truth global positions in prediction. # tensor: (1, num_peds, 2, 12)
        # 12 means pred_period.

        - obs_traj_rel # displacement in observation. # tensor: (1, num_peds, 2, 8)
        # obs_traj_rel[:,:,:,0] is zero. # obs_traj_rel[:,:,:,1:]=obs_traj[:,:,:,1:]-obs_traj[:,:,:,:-1]

        - pred_traj_gt_rel # ground truth displacement in prediction. # tensor: (1, num_peds, 2, 12)
        # pred_traj_gt_rel[:,:,:,0]=pred_traj_gt[:,:,:,0]-obs_traj[:,:,:,-1]

        - non_linear_ped # 0-1 vector indicates whether motion is nonlinear. # tensor: (1, num_peds)

        - loss_mask # all-one tensor. # tensor: (1, num_peds, 20) # 20 means full_period.

        - V_obs # vertices represent displacement of pedestrians in observation.
        # tensor: (1, 8, num_peds, 2) # V_obs.permute(0,2,3,1) == obs_traj_rel

        - A_obs # Adjacency matrix of pedestrians in observation. # tensor: (1, 8, num_peds, num_peds)

        - V_tr # ground truth vertices represent displacement of pedestrians in prediction.
        # tensor: (1, 12, num_peds, 2)

        - A_tr # ground truth Adjacency matrix of pedestrians in prediction.
        # tensor: (1, 12, num_peds, num_peds)
    """
    pass

In [ ]:
### expected output: ###
### Visualization on trajectories and attention ###

visualize_dataloader(dloader_train, attn_scale=1.5, device=device)

## 4. Train and evaluate a [Social-STGCNN](https://arxiv.org/pdf/2002.11927.pdf) model on a multi-pedestrian trajectory prediction task.

![title](social-stgcnn.png)

In [ ]:
# Run the training file to train a Social-STGCNN model.

!python torch_data_tutorial/train.py --num_epochs 100 --lr 0.01 --dataset zara1 --attn_mech auth --n_stgcnn 1 --n_txpcnn 5 --use_lrschd && echo "Training on zara1 is complete."

In [ ]:
# Run the test on the model you just trained.

!python torch_data_tutorial/test.py

In [ ]:
# Run the test on the model I trained using 200 epochs in this notebook.

!python torch_data_tutorial/test.py --pretrained

In [ ]:
# Run the test on the model provided by the original github repo.

!python torch_data_tutorial/test.py --original

In [ ]:
# Visualize prediction from the model and compare against ground truth.
import os
import math
import sys
sys.path.append('.')
pkg_path = 'torch_data_tutorial'
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle
import glob
import matplotlib.pyplot as plt
import torch.distributions.multivariate_normal as torchdist
from torch_data_tutorial.utils import load_dataset
from torch_data_tutorial.metrics import ade, fde, seq_to_nodes, nodes_rel_to_nodes_abs
from torch_data_tutorial.model import social_stgcnn
import copy

### Select your model starts ###
paths = ['torch_data_tutorial/checkpoint/auth-zara1-pretrained']
# paths = ['torch_data_tutorial/checkpoint/auth-zara1']
# paths = ['torch_data_tutorial/checkpoint/social-stgcnn-zara1']
### Select your model ends ###

def test(KSTEPS=20):
    global loader_test,model
    model.eval()
    ade_bigls = []
    fde_bigls = []
    raw_data_dict = {}
    step =0 
    for batch in loader_test: 
        step+=1
        #Get data
        batch = [tensor.cuda() for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
            loss_mask,V_obs,A_obs,V_tr,A_tr = batch
        num_of_objs = obs_traj_rel.shape[1]
        V_obs_tmp =V_obs.permute(0,3,1,2)
        V_pred,_ = model(V_obs_tmp,A_obs.squeeze())
        V_pred = V_pred.permute(0,2,3,1)
        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        V_pred = V_pred.squeeze()
        num_of_objs = obs_traj_rel.shape[1]
        V_pred,V_tr =  V_pred[:,:num_of_objs,:],V_tr[:,:num_of_objs,:] 
        sx = torch.exp(V_pred[:,:,2]) #sx
        sy = torch.exp(V_pred[:,:,3]) #sy
        corr = torch.tanh(V_pred[:,:,4]) #corr
        cov = torch.zeros(V_pred.shape[0],V_pred.shape[1],2,2).cuda()
        cov[:,:,0,0]= sx*sx
        cov[:,:,0,1]= corr*sx*sy
        cov[:,:,1,0]= corr*sx*sy
        cov[:,:,1,1]= sy*sy
        mean = V_pred[:,:,0:2]
        mvnormal = torchdist.MultivariateNormal(mean,cov)
        ### Rel to abs
        #Now sample 20 samples
        ade_ls = {}
        fde_ls = {}
        V_x = seq_to_nodes(obs_traj.data.cpu().numpy().copy()) # obs
        V_x_rel_to_abs = nodes_rel_to_nodes_abs(V_obs.data.cpu().numpy().squeeze().copy(),
                                                 V_x[0,:,:].copy())
        V_y = seq_to_nodes(pred_traj_gt.data.cpu().numpy().copy()) # target
        V_y_rel_to_abs = nodes_rel_to_nodes_abs(V_tr.data.cpu().numpy().squeeze().copy(),
                                                 V_x[-1,:,:].copy())
        raw_data_dict[step] = {}
        raw_data_dict[step]['obs'] = copy.deepcopy(V_x_rel_to_abs)
        raw_data_dict[step]['trgt'] = copy.deepcopy(V_y_rel_to_abs)
        raw_data_dict[step]['pred'] = []
        for n in range(num_of_objs): # num_peds
            ade_ls[n]=[]
            fde_ls[n]=[]
        for k in range(KSTEPS):
            fig_no_pred, ax_no_pred = plt.subplots()
            fig, ax = plt.subplots()
            V_pred = mvnormal.sample()
            V_pred_rel_to_abs = nodes_rel_to_nodes_abs(V_pred.data.cpu().numpy().squeeze().copy(),
                                                     V_x[-1,:,:].copy())
            raw_data_dict[step]['pred'].append(copy.deepcopy(V_pred_rel_to_abs))
           # print(V_pred_rel_to_abs.shape) #(12, 3, 2) = seq, ped, location
            for n in range(num_of_objs):
                pred = [] 
                target = []
                obsrvs = [] 
                number_of = []
                pred.append(V_pred_rel_to_abs[:,n:n+1,:])
                target.append(V_y_rel_to_abs[:,n:n+1,:])
                obsrvs.append(V_x_rel_to_abs[:,n:n+1,:])
                number_of.append(1)
                ### visualization starts ###
                vis_obs = V_x_rel_to_abs[:,n] # (8, 2)
                vis_target = target[-1][:,0] # (12, 2)
                vis_pred = pred[-1][:,0] # (12, 2)
                vis_obs_target = np.concatenate([vis_obs, vis_target], axis=0)
                vis_obs_pred = np.concatenate([vis_obs, vis_pred], axis=0)
                ax.plot(vis_obs_target[:,0], vis_obs_target[:,1], 'o-',c='C3')
                ax.plot(vis_obs_pred[:,0], vis_obs_pred[:,1], 'o-',c='k')
                ax.plot(vis_obs[:,0], vis_obs[:,1], 'o-',c='C0')
                ax_no_pred.plot(vis_obs_target[:,0], vis_obs_target[:,1], 'o-',c='C3')
                ax_no_pred.plot(vis_obs[:,0], vis_obs[:,1], 'o-',c='C0')
                ### visualization ends ###
                ade_ls[n].append(ade(pred,target,number_of))
                fde_ls[n].append(fde(pred,target,number_of))
            ax.set_aspect('equal', adjustable='box')
            ax_no_pred.set_aspect('equal', adjustable='box')
            plt.show()
            break
            
        for n in range(num_of_objs):
            ade_bigls.append(min(ade_ls[n]))
            fde_bigls.append(min(fde_ls[n]))
        
        break

    ade_ = sum(ade_bigls)/len(ade_bigls)
    fde_ = sum(fde_bigls)/len(fde_bigls)
    return ade_,fde_,raw_data_dict

KSTEPS=5
print("*"*50)
print('Number of samples:',KSTEPS)
print("*"*50)

for feta in range(len(paths)):
    ade_ls = [] 
    fde_ls = [] 
    path = paths[feta]
    exps = glob.glob(path)
    print('Model being tested are:',exps)

    for exp_path in exps:
        print("*"*50)
        print("Evaluating model:",exp_path)

        model_path = exp_path+'/val_best.pth'
        args_path = exp_path+'/args.pkl'
        with open(args_path,'rb') as f: 
            args = pickle.load(f)

        args.attn_mech = 'auth' # force attn_mech when original implementation is used.

        stats= exp_path+'/constant_metrics.pkl'
        with open(stats,'rb') as f: 
            cm = pickle.load(f)
        print("Stats:",cm)
        #Data prep     
        obs_seq_len = args.obs_seq_len
        pred_seq_len = args.pred_seq_len
        loader_test = load_dataset(args, pkg_path, subfolder='test', num_workers=1)
        #Defining the model 
        model = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
        output_feat=args.output_size,seq_len=args.obs_seq_len,
        kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len).cuda()
        model.load_state_dict(torch.load(model_path))
        print("Testing ....")
        ad,fd,raw_data_dic_= test()
